# Stock Analysis and Prediction

This notebook contains tools for analyzing stock data and making predictions.

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Set visualization styles
try:
    plt.style.use("seaborn-v0_8-darkgrid")
except:
    plt.style.use("seaborn-darkgrid")  # Fallback
sns.set_theme(style="darkgrid")

ModuleNotFoundError: No module named 'matplotlib'

## Data Collection

We'll use Yahoo Finance to fetch stock data.

In [11]:
# Define stock symbol and date range
stock_symbol = '^NSEI'  # Apple Inc.
start_date = datetime.now() - timedelta(days=365*5)  # 5 years of data
end_date = datetime.now()

# Fetch data
stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

# Display the first few rows
stock_data.head()

NameError: name 'datetime' is not defined

## Data Visualization

Let's visualize the stock price history.

In [ ]:
# Plot closing price
plt.figure(figsize=(14, 7))
plt.plot(stock_data['Close'])
plt.title(f'{stock_symbol} Stock Price History')
plt.xlabel('Date')
plt.ylabel('Close Price (USD)')
plt.grid(True)
plt.show()

## Feature Engineering

Let's create some features for our prediction model.

In [ ]:
# Add rolling mean features
stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()
stock_data['MA200'] = stock_data['Close'].rolling(window=200).mean()

# Add price momentum (daily returns)
stock_data['Returns'] = stock_data['Close'].pct_change()

# Add volatility (rolling standard deviation)
stock_data['Volatility'] = stock_data['Returns'].rolling(window=20).std()

# Drop NaN values
stock_data = stock_data.dropna()

# Display the data with new features
stock_data.head()

## Prediction Model

Create a simple linear regression model to predict future stock prices.

In [ ]:
# Create features and target
# Predict next day's closing price
stock_data['Target'] = stock_data['Close'].shift(-1)
stock_data = stock_data.dropna()

# Select features
features = ['Close', 'MA50', 'MA200', 'Returns', 'Volatility']
X = stock_data[features]
y = stock_data['Target']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error: {mse:.2f}')
print(f'R-squared: {r2:.2f}')

## Visualization of Predictions

Compare actual vs predicted values.

In [ ]:
# Create a DataFrame with actual and predicted values
results = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
results = results.sort_index()

# Plot actual vs predicted
plt.figure(figsize=(14, 7))
plt.plot(results.index, results['Actual'], label='Actual')
plt.plot(results.index, results['Predicted'], label='Predicted', alpha=0.7)
plt.title(f'{stock_symbol} - Actual vs Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

## Future Predictions

Predict the stock price for the next trading day.

In [ ]:
# Get the latest data point
latest_data = stock_data[features].iloc[-1].values.reshape(1, -1)

# Make prediction for the next day
next_day_prediction = model.predict(latest_data)[0]

print(f'Last Close Price: ${stock_data["Close"].iloc[-1]:.2f}')
print(f'Predicted Next Day Price: ${next_day_prediction:.2f}')
print(f'Predicted Change: {((next_day_prediction / stock_data["Close"].iloc[-1]) - 1) * 100:.2f}%')